In [1]:
#==DeepFM==
# 一、背景
# CTR问题，被证明的最有效的提升任务表现的策略是特征组合(Feature Interaction),
# 如何更好地学习特征组合，进而更加精确地描述数据的特点。是基础推荐模型到深度学习推荐模型遵循的一个主要的思想。
# 如何更高效的学习特征组合？二阶升三阶、四阶，复杂度就成几何倍的升高。
# 为了解决上述问题，出现了FM和FFM来优化LR的特征组合较差这一个问题。
# DNN在特征组合方面有优势，所以又出现了FNN和PNN等使用深度网络的模型。但是DNN也存在局限性。
# ==DNN局限== 使用DNN网络解决推荐问题 存在网络参数过于庞大的问题，
# 这是因为在进行特征处理的时候我们需要使用one-hot编码来处理离散特征，导致输入的维度猛增。
# 为了解决DNN参数量过大的局限性，可以采用非常经典的Field思想，将OneHot特征转换为Dense Vector 稠密矩阵。
# 思想是避免全连接，分而治之。
# 后面再增加全连接层就可以实现高阶的特征组合。
# 但是仍然缺少低阶的特征组合，于是增加FM来表示低阶的特征组合。
# ==FNN和PNN==
# 结合FM和DNN的方式：FNN、DeepFM、NFM
# FNN是使用预训练好的FM模块，得到隐向量，然后把隐向量作为DNN的输入，但是经过实验进一步发现，在Embedding layer和hidden layer1之间增加一个product层（如上图所示）可以提高模型的表现，所以提出了PNN，使用product layer替换FM预训练层。
# ==Wide&Deep DeepFM==
# FNN和PNN模型仍然有一个比较明显的尚未解决的缺点：对于低阶组合特征学习到的比较少，这一点主要是由于FM和DNN的串行方式导致的，也就是虽然FM学到了低阶特征组合，但是DNN的全连接结构导致低阶特征并不能在DNN的输出端较好的表现。
# 将串行方式改进为并行方式能比较好的解决这个问题。
# Wide&Deep的构成方式，虽然将整个模型的结构调整为了并行结构,实际的使用中Wide Module中的部分需要较为精巧的特征工程,人工参与的成份比较大。
# 它的问题已清晰的定位到，在output Units阶段直接将低阶和高阶特征进行组合，很容易让模型最终偏向学习到低阶或者高阶的特征，而不能做到很好的结合。
# 所以DeepFM 是将Wide&Deep的Wide部分替换成了FM。

# 二、DeepFM模型的结构与原理

# 前面的Field和Embedding处理是和前面的方法是相同的。
# 后面DeepFM将Wide部分替换为了FM 层。
# 2.1模型中的FM层
# FM Layer是由一阶特征和二阶特征Concatenate到一起在经过一个Sigmoid得到logits（结合FM的公式一起看），所以在实现的时候需要单独考虑linear部分和FM交叉特征部分。
# //Dense层结果经过FM做交叉？以解决纯Wide&Deep中只到最后一层组合的尴尬。
# 2.2模型中的Deep层
# Deep Module是为了学习高阶的特征组合，在上图中使用用全连接的方式将Dense Embedding输入到Hidden Layer，这里面Dense Embeddings（将One-hot稀疏变稠密）就是为了解决DNN中的参数爆炸问题，这也是推荐模型中常用的处理方法。
# 后面将所有id类特征对应的embedding向量concat到到一起输入到DNN中。
# 最后进入DNN部分输出使用sigmod激活函数进行激活。


In [2]:
# 三、代码实现
def DeepFM(linear_feature_columns, dnn_feature_columns):
    # 构建输入层，即所有特征对应的Input()层，这里使用字典的形式返回，方便后续构建模型
    dense_input_dict, sparse_input_dict = build_input_layers(linear_feature_columns + dnn_feature_columns)

    # 将linear部分的特征中sparse特征筛选出来，后面用来做1维的embedding
    linear_sparse_feature_columns = list(filter(lambda x: isinstance(x, SparseFeat), linear_feature_columns))

    # 构建模型的输入层，模型的输入层不能是字典的形式，应该将字典的形式转换成列表的形式
    # 注意：这里实际的输入与Input()层的对应，是通过模型输入时候的字典数据的key与对应name的Input层
    input_layers = list(dense_input_dict.values()) + list(sparse_input_dict.values())

    # linear_logits由两部分组成，分别是dense特征的logits和sparse特征的logits
    linear_logits = get_linear_logits(dense_input_dict, sparse_input_dict, linear_sparse_feature_columns)

    # 构建维度为k的embedding层，这里使用字典的形式返回，方便后面搭建模型
    # embedding层用户构建FM交叉部分和DNN的输入部分
    embedding_layers = build_embedding_layers(dnn_feature_columns, sparse_input_dict, is_linear=False)

    # 将输入到dnn中的所有sparse特征筛选出来
    dnn_sparse_feature_columns = list(filter(lambda x: isinstance(x, SparseFeat), dnn_feature_columns))

    fm_logits = get_fm_logits(sparse_input_dict, dnn_sparse_feature_columns, embedding_layers) # 只考虑二阶项

    # 将所有的Embedding都拼起来，一起输入到dnn中
    dnn_logits = get_dnn_logits(sparse_input_dict, dnn_sparse_feature_columns, embedding_layers)
    
    # 将linear,FM,dnn的logits相加作为最终的logits
    output_logits = Add()([linear_logits, fm_logits, dnn_logits])

    # 这里的激活函数使用sigmoid
    output_layers = Activation("sigmoid")(output_logits)

    model = Model(input_layers, output_layers)
    return model

In [3]:
# #代码解释
# 模型大致由两部分组成：FM，DNN
# 可以将整个模型拆成三部分，分别是一阶特征处理linear部分，二阶特征交叉FM以及DNN的高阶特征交叉
# //此外每一部分可能由是由不同的特征组成，所以在构建模型的时候需要分别对这三部分输入的特征进行选择。
# inear_logits: 
#     这部分是有关于线性计算，也就是FM的前半部分,
#     用了一个get_linear_logits函数实现,
#     得到linear的输出， 这部分特征由数值特征和类别特征的onehot编码组成的一维向量组成，实际应用中根据自己的业务放置不同的一阶特征.
# fm_logits:
#     这一块主要是针对离散的特征，首先过embedding，然后使用FM特征交叉的方式，两两特征进行交叉，得到新的特征向量，最后计算交叉特征的logits
# dnn_logits:
#     这一块主要是针对离散的特征，首先过embedding，然后将得到的embedding拼接成一个向量,通过dnn学习类别特征之间的隐式特征交叉并输出logits值.
    

In [ ]:
# #四、思考
# 1、如果对于FM采用随机梯度下降SGD训练模型参数，请写出模型各个参数的梯度和FM参数训练的复杂度
# SGD参数：keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False) 
# lr: float >= 0. 学习率。
# momentum: float >= 0. 参数，用于加速 SGD 在相关方向上前进，并抑制震荡。
# decay: float >= 0. 每次参数更新后学习率衰减值。
# nesterov: boolean. 是否使用 Nesterov 动量。
    
# 2、对模型图中，根据你的理解Sparse Feature中的不同颜色节点分别表示什么意思
# Dense中的灰色是将原始特征稠密化处理后的特征，
# 黄色部分FM中是代表有一阶的，有二阶交叉的特征，
# DEEP部分黄色有对号的的是Dense送到DNN里的特征。最底端黄色弯线的是与FM结合后的最终特征。
